In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
tweets = pd.read_csv("data/translated_tweets.csv.gz")
tweets['SentimentText'] = tweets['SentimentText'].str.lower().str.replace(r'[^\w\s]', '')

In [4]:
tweets.drop("SentimentSource", axis="columns", inplace=True)
tweets.head()

,ItemID,Sentiment,SentimentText
0,1,0,je to tak smutné pre môjho apl priateľa..........
1,2,0,chýbal mi nový trailer...
2,3,1,omg je už 7:30 :o
3,4,0,.. omgaga. im sooo im gunna cry. som bol u toh...
4,5,0,"myslím si, že mi bf podvádza na mňa!!!"


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(tweets['SentimentText'].values)
X = tokenizer.texts_to_sequences(tweets['SentimentText'].values)
X = pad_sequences(X)

2024-06-22 17:18:38.781345: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 17:18:38.801815: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 17:18:39.220730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ebozik/anaconda3/envs/nlp2/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [6]:
Y = tweets['Sentiment'].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.models import Sequential
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/home/ebozik/anaconda3/envs/nlp2/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-06-22 17:20:04.955315: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-22 17:20:04.970543: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-22 17:20:04.970690: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-22 17:20:04.972900: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
batch_size = 32
epochs = 5

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), callbacks=[early_stopping])
# model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), callbacks=[early_stopping])

Epoch 1/5
   13/28453 ━━━━━━━━━━━━━━━━━━━━ 3:17:20 416ms/step - accuracy: 0.6568 - loss: 0.6319

KeyboardInterrupt: 

In [24]:
model.save("trained_lstm.keras")

In [25]:
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print("Test score:", score)
print("Test accuracy:", acc)

7114/7114 - 372s - 52ms/step - accuracy: 0.7866 - loss: 0.4525
Test score: 0.4525470733642578
Test accuracy: 0.7865970134735107


In [49]:
def predict_sentiment(text):
    text = text.lower().replace(r'[^\w\s]', '')
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=X.shape[1])
    pred = model.predict(padded)
    return 'Positive' if np.argmax(pred) == 1 else 'Negative'

In [50]:
texts = [
    "Akokoľvek sa na to pozriem tak to nevyzerá dobre.",
    "Tento deň je skvelý",
    "Neviem či som správne pochopil.",
    "To si vážne urobila?!",
    "To myslíš vážne?",
    "Ja som veľmi rád, že si sa to konečne naučila.",
    "Som pekný.",
    "Zajtra si kúpim pivo.",
    "Tak teraz neviem čo ďalej.",
    "Prehrali sme ale vidím na tom aj pozitívum.",
    "Tak som rozmýšlal, že to asi aj výjde."
]
for i in texts:
    print(predict_sentiment(i))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Positive


In [45]:
import pickle 
with open("tokenizers/tweets_lstm_tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
os.getcwd()

'/mnt/c/Users/Erik/Projects/nlp_sentiment_analysis'

In [46]:
X.shape[1]

255